In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense, concatenate

# Make NumPy printouts easier to read.
np.set_printoptions(precision=5, suppress=True)

import tensorflow as tf
print(tf.__version__)

2.13.0


In [18]:
dataset = pd.read_csv('data_obrobiona/calosc.csv')

dataset

,dzien_tygodnia,czas,przeplyw
0,4,0.009907,19.7500
1,4,0.019815,8.8750
2,4,0.029722,19.8125
3,4,0.039629,16.0000
4,4,0.049537,4.5000
...,...,...,...
369504,2,99.980900,17.3125
369505,2,99.985600,19.8125
369506,2,99.990400,15.6250
369507,2,99.995200,4.5000


In [19]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
x_train=train_dataset.drop(columns="przeplyw");
x_test=test_dataset.drop(columns="przeplyw");
y_train=train_dataset.przeplyw;
y_test=test_dataset.przeplyw;
train_data=x_train.drop(columns="czas")
train_czas=x_train.drop(columns="dzien_tygodnia")

In [20]:
train_czas.shape

(295607, 1)

In [21]:
model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(126, activation='relu', input_shape=(2,)),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1,activation='linear') 
])

In [22]:


model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
              loss="mse",
              metrics=['mae'])

In [23]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
9238/9238 [==============================] - 7s 731us/step - loss: 21.8988 - mae: 4.0315
Epoch 2/5
9238/9238 [==============================] - 7s 715us/step - loss: 21.3336 - mae: 3.9958
Epoch 3/5
9238/9238 [==============================] - 7s 732us/step - loss: 21.3268 - mae: 3.9962
Epoch 4/5
9238/9238 [==============================] - 6s 697us/step - loss: 21.3194 - mae: 3.9955
Epoch 5/5
9238/9238 [==============================] - 6s 687us/step - loss: 21.3169 - mae: 3.9956


In [24]:
model.evaluate(x_test,  y_test, verbose=2)

2310/2310 - 1s - loss: 21.6090 - mae: 4.0211 - 1s/epoch - 561us/step


[21.608963012695312, 4.021121501922607]

In [25]:
sample_time=2;      #[s]
symulation_time=60; #[min]

current_hours=12;   #[h]
current_minutes=00; #[min]
current_day=1;      #[label, pon-1, wt-2,...,nd-7]

In [26]:
sample_time_normalized=sample_time/(60*60*24);              #[min - normalized]
symulation_time_normlized=symulation_time/(24*60)           #[min - normalized]
current_time=(current_hours*60+current_minutes)/(24*60)     #[min - normalized]

endtime=current_time+symulation_time_normlized;
if endtime<1:
    flow_time=np.arange(current_time,endtime,sample_time_normalized)                    #[min - normalized]
    flow_time=np.vstack((current_day*np.ones_like(flow_time, dtype=int),flow_time)).T   #[label, min]
else:
    temp_1=np.arange(current_time,1,sample_time_normalized)                             #[min - normalized]
    temp_1=np.vstack((current_day*np.ones_like(temp_1, dtype=int),temp_1)).T            #[label, min]

    temp_2=np.arange(0,endtime-1,sample_time_normalized)                                #[min - normalized]
    temp_2=np.vstack(((current_day+1)*np.ones_like(temp_2, dtype=int),temp_2)).T        #[label, min] 

    flow_time=np.concatenate((temp_1,temp_2))                                           #[label, min]
flow_time

array([[1.     , 0.5    ],
       [1.     , 0.50002],
       [1.     , 0.50005],
       ...,
       [1.     , 0.5416 ],
       [1.     , 0.54162],
       [1.     , 0.54164]])

In [27]:
flow_result = model.predict(flow_time)                   # [l/min]
water_usage_predict=np.sum(flow_result)*(sample_time/60) # [l]

water_usage_predict

57/57 [==============================] - 0s 538us/step


693.7286458333333

In [28]:
flow_result

array([[11.56214],
       [11.56214],
       [11.56214],
       ...,
       [11.56214],
       [11.56214],
       [11.56214]], dtype=float32)